In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
train_path = 'D:/DLcompetition/data/train'
test_path = 'D:/DLcompetition/data/test'

# Model For Scene A

## Four tables in total：

* train_behavior_A.csv: consumer table
  * 21245 ccx_id, each ccx_id corresponds to a entry (a customer)
  * 2270 labels: var1-var2270
  * var19 is time
* train_consumer_A.csv: behavior table
  * 268453 entries, each ccx_id corresponds to multiple entries (multiple behaviors)
  * 14 labels: V_1-V_14
  * V_7 and V_11 are time
* train_ccx_A.csv: record history
  * 53983 entries, each ccx_id corresponds to multiple entries（multiple records）
  * 6 labels: var_01-var_06
* train_target_A.csv: labels
  * 21245 ccx_id, each ccx_id corresponds to one target

# Processing for Scene A

## 1. ccx_A history record

In [3]:
train_ccx_A = pd.read_csv(train_path + '/' + 'train_ccx_A.csv', index_col=0, parse_dates = ["var_06"])
test_ccx_A = pd.read_csv(test_path + '/' + 'test_ccx_A.csv', index_col=0, parse_dates = ["var_06"])
all_ccx = pd.concat([train_ccx_A, test_ccx_A])
# print(all_ccx.isnull().sum()) no N/A in ccx

In [4]:
# Handle strings.
# Create columns for each different string value. The values under the columns are number of appearances for each id.
def Clean_history(data, column):
    # make a copy to facilitate counting
    data = data.copy()
    data['zeros'] = np.zeros(len(data))
    group = data.reset_index()[['zeros', 'ccx_id', column]].groupby(['ccx_id', column])
    data = group.count().unstack([column])['zeros'].fillna(0)
    data.columns = [column + '_' + data.columns[i] + '_count' for i in range(len(data.columns))]
    return data

In [5]:
ccx_master = pd.concat([Clean_history(all_ccx, column) for column in all_ccx.columns[:5]], axis=1)

In [6]:
# For the date column, we extra four different columns
# 1. The time range under an id
# 2. How many different dates under one id
# 3. Frequecy: column 2 divided by column 1
# 4. Average number of records per id
def Date_transform(data, column):
    group = (data[column] - min(data[column])).groupby('ccx_id')
    different_days_num = data.reset_index().groupby(["ccx_id", column]).first().reset_index()[['ccx_id', column]].groupby('ccx_id').count()
    date_length = group.max().astype('timedelta64[D]').astype(int) + 1
    date_info = pd.concat([date_length, different_days_num, different_days_num / pd.DataFrame(date_length), pd.DataFrame(group.count()) / different_days_num], axis=1)
    date_info.columns = [column + '_' + item for item in ['daylength', 'numdays', 'frequencies', 'numperday']]
    return date_info

In [7]:
ccx_master = ccx_master.join(Date_transform(all_ccx, 'var_06'))

## 2. consumer_A consumer information data

In [8]:
train_consumer_A = pd.read_csv(train_path + '/' + 'train_consumer_A.csv', index_col=0)
test_consumer_A = pd.read_csv(test_path + '/' + 'test_consumer_A.csv', index_col=0)
all_consumer = pd.concat([train_consumer_A, test_consumer_A])

In [9]:
# Replace the outlier with nan
index = all_consumer[all_consumer['V_12'] == max(all_consumer['V_12'])].index[0]
all_consumer['V_12'][index] = np.nan

c:\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
# For the mode of a column which is of type string for each id
def String_mode(data, column):
    counts = data[[column]].reset_index().reset_index().groupby(['ccx_id', column]).count()
    mode = counts.reset_index().set_index(['ccx_id', 'index']).sort_index(ascending=False).groupby('ccx_id').first()
    mode = mode.rename(columns={"index": column})
    return mode

# numeric column. replace null with median
medians = pd.concat([all_consumer[['V_' + num]].groupby('ccx_id').median() for num in ['12', '13']], axis=1)
all_consumer = all_consumer.fillna(medians.loc[all_consumer.index].fillna(medians.median()))
# string type column , replace null with mode
modes = pd.concat([String_mode(all_consumer, 'V_' + num) for num in ['8', '14']], axis=1)
all_consumer = all_consumer.fillna(modes.loc[all_consumer.index].fillna(modes.mode().iloc[0]))
# How many records per id.
consumer_count = all_consumer['V_1'].reset_index().groupby('ccx_id').count().rename(columns={"V_1": "consumer_count"})
# 对V_1, V_2, V_3三列进行组合，数出每个id有多少个城市
# For V_1, V_2, V_3, count the number of cities for each id.
places_count = all_consumer[['V_1', 'V_2', 'V_3']].reset_index().reset_index().groupby(['ccx_id', 'V_1', 'V_2', 'V_3']).count().groupby('ccx_id').count().rename(columns={"index": "places_count"})

In [13]:
# For numeric column, calculate min, max, median, and average of each id
def Num_transform(data, column):
    group = all_consumer[['V_4']].groupby('ccx_id')
    transformed = pd.concat([group.min(), group.median(), group.max(), group.mean(), group.std().fillna(0)], axis=1)
    transformed.columns = [column + '_' + item for item in ['min', 'median', 'max', 'mean', 'std']]
    return transformed

In [14]:
transformed_numbers = pd.concat([Num_transform(all_consumer, 'V_' + str(i)) for i in [4, 5, 6, 9, 10, 12, 13]], axis=1)
transformed_strings = pd.concat([Clean_history(all_consumer, 'V_' + num) for num in ['8', '14']], axis=1)
# For V_7, count the number of dates per id.
distinct_timecount = all_consumer[['V_7']].reset_index().reset_index().groupby(['ccx_id', 'V_7']).count().groupby('ccx_id').count().rename(columns = {"index": "V_7"})
# For V_11，count the number of rows that has value "0000-00-00 00:00:00"
zerotime_count = all_consumer[all_consumer['V_11'] == '0000-00-00 00:00:00'][['V_11']].groupby('ccx_id').count()

In [18]:
consumer_master = pd.concat([consumer_count, places_count, transformed_numbers, transformed_strings, 
                             distinct_timecount, zerotime_count], axis=1).fillna(0)

## 3. behavior_A Consumer table

In [19]:
train_behavior_A = pd.read_csv(train_path + '/' + 'train_behavior_A.csv', index_col=0)
test_behavior_A = pd.read_csv(test_path + '/' + 'test_behavior_A.csv', index_col=0)
all_behavior = pd.concat([train_behavior_A, test_behavior_A])

c:\python36\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
# Fill out the null values, and calculate the time range between var17 and var16
all_behavior['var10'] = all_behavior['var10'].fillna(0)
all_behavior['yearlength'] = all_behavior['var17'] - all_behavior['var16']

In [21]:
import sklearn.preprocessing as prep

def Cat_to_bin(data, a = 0.01):
    '''Transfrom a categorical column to onehotencoding'''
    tmp = pd.value_counts(data)/data.shape[0]
    cat = list(tmp.index[tmp > a])
    enc = prep.OneHotEncoder(n_values = len(cat)+1, sparse = False)
    xbin = enc.fit_transform(np.transpose(
            [data.astype("category").cat.set_categories(cat).cat.rename_categories(1+np.arange(len(cat))).astype("float").fillna(0).values]))[:,1:]     
    dabin = pd.DataFrame(xbin, columns = ["{}_{}".format(data.name, x) for x in cat], index = data.index)
    if(tmp[tmp <= a].sum() > a):
        dabin = pd.concat([dabin, pd.DataFrame({"{}_Others".format(data.name):data.notnull()-dabin.sum(axis = 1)})], axis = 1)
    if(dabin.shape[1] == 2):
        dabin = pd.DataFrame({data.name: xbin[:,0]}, index = data.index)
    return(dabin)

In [23]:
data_bin = pd.concat([Cat_to_bin(all_behavior['var' + str(i)]) for i in [3, 4, 5, 6, 11, 12, 13, 14, 15, 18, 19]], axis=1)

In [24]:
behavior_master = pd.concat([all_behavior.drop(['var' + str(i) for i in [3, 4, 5, 6, 11, 12, 13, 14, 15, 18, 19]], axis=1), 
                             data_bin], axis=1)

## 4. Merge

In [25]:
master = pd.concat([behavior_master, consumer_master, ccx_master], axis=1)

In [26]:
def Value_counts(data, nhead = 5):
    tmp = pd.value_counts(data).reset_index().rename_axis({"index": data.name}, axis = 1)
    value = pd.DataFrame(['value {}'.format(x+1) for x in range(nhead)], index = np.arange(nhead)).join(tmp.iloc[:,0], how = "left").set_index(0).T
    freq = pd.DataFrame(['freq {}'.format(x+1) for x in range(nhead)], index = np.arange(nhead)).join(tmp.iloc[:,1], how = "left").set_index(0).T
    nnull = data.isnull().sum()
    freqother = pd.DataFrame({data.name: [data.shape[0]-nnull-np.nansum(freq.values), nnull]}, index = ["freq others", "freq NA"]).T
    counts = pd.concat([value, freq, freqother], axis = 1)
    if 'level_0' in counts.index:
        print(data.name)
    return(counts)

tmp = pd.concat(map(lambda i: Value_counts(master.loc[:,i]), master.columns))
# For each column, if the number of null values and the number of modes
# account for over 99.9% of this column, then drop it 
master = master.loc[:, (tmp["freq 1"] + tmp["freq NA"])/master.shape[0] < 0.999]

In [29]:
# drop columns that has correlation over 0.99
master = master.drop(master.columns[np.any(np.abs(np.tril(np.corrcoef(master.rank(pct = True).fillna(0.5).values, rowvar = 0), -1)) > 0.99, axis = 0)], axis = 1)

In [30]:
for column in master.columns:
    if column in consumer_master or column in ccx_master:
        master[column] = master[column].fillna(0)

master = master.fillna(0)

In [32]:
train_target_A = pd.read_csv(train_path + '/' + 'train_target_A.csv', index_col=0)
master = master.join(train_target_A)

In [53]:
master.to_csv('master.csv')